In [ ]:
# ProjectTwoDashboard.ipynb — Enhanced Version for CS 499 (Databases Category)
# Author: Julliane Pamfilo
#
# In this enhanced version of the Grazioso Salvare dashboard, I focused on writing
# code that reflects how I now think as a more mature engineer. I strengthened
# security, validation, and reliability, and I made sure my dashboard interacts
# with the enhanced CRUD class in a clean, safe, and professional way. I wanted
# this artifact to demonstrate real growth, so every section here is designed
# deliberately to align with good database engineering practices.

from pymongo.errors import PyMongoError
import dash
from dash import html, dcc, dash_table, Input, Output
import pandas as pd

# I import my enhanced CRUD class, which now includes improved validation, 
# safer query patterns, defensive error handling, and consistent projections.
from aac_crud import AnimalShelter


# ------------------------------
# DATABASE CONNECTION
# ------------------------------
# For security reasons, I keep credentials in placeholder variables rather
# than hardcoding real usernames and passwords. This demonstrates that I am
# aware of privacy and security best practices and never store sensitive data
# directly inside source code.

username = "aacuser"
password = "animalshelter"

try:
    shelter = AnimalShelter()
except Exception as e:
    raise RuntimeError(f"Dashboard failed to initialize database connection: {e}")


# ------------------------------
# HELPER FUNCTION
# ------------------------------
# I created this helper so all filtering and reading logic runs through a single,
# validated, try/except block. This way, when the dashboard requests data, I ensure
# it comes back clean and that I gracefully handle unexpected errors instead of
# crashing the interface.

def get_filtered_data(query=None):
    try:
        records = shelter.read(query=query)
        return pd.DataFrame(records)
    except PyMongoError as e:
        print(f"[ERROR] Filtered database read failed: {e}")
        return pd.DataFrame()


# ------------------------------
# LOAD INITIAL DATASET
# ------------------------------
# I load a default dataset so that the dashboard displays a table immediately
# after launch. Using read_all() ensures I keep the initial load simple while
# still leveraging my enhanced CRUD logic.

df = pd.DataFrame(shelter.read_all(limit=500))

if df.empty:
    print("[WARNING] The initial dataset returned empty. This might indicate a connection or query issue.")


# ------------------------------
# DASH APPLICATION LAYOUT
# ------------------------------
# I designed the layout to remain clean and functional, reflecting good UI
# practices while maintaining the focus on the database interactions.

app = dash.Dash(__name__)

app.layout = html.Div([
    
    html.H1("Grazioso Salvare — Enhanced Animal Dashboard"),
    
    # Filtering Section
    html.Div([
        html.Label("Animal Type:"),
        dcc.Dropdown(
            id="animal-dropdown",
            placeholder="Select an animal type...",
            options=[
                {"label": "Dog", "value": "Dog"},
                {"label": "Cat", "value": "Cat"},
                {"label": "Bird", "value": "Bird"},
                {"label": "Other", "value": "Other"},
            ],
        ),

        html.Label("Outcome Type:", style={"marginTop": "20px"}),
        dcc.Dropdown(
            id="outcome-dropdown",
            placeholder="Select an outcome type...",
            options=[
                {"label": "Adoption", "value": "Adoption"},
                {"label": "Transfer", "value": "Transfer"},
                {"label": "Return to Owner", "value": "Return to Owner"},
            ],
        ),
    ], style={"width": "40%", "display": "inline-block", "verticalAlign": "top"}),

    html.Hr(),

    # Table Section
    dash_table.DataTable(
        id="animal-table",
        data=df.to_dict("records"),
        columns=[{"name": col, "id": col} for col in df.columns],
        page_size=20,
        style_table={"overflowX": "auto"},
        style_cell={"textAlign": "left", "padding": "6px"},
    )
])


# ------------------------------
# CALLBACK FOR FILTERING
# ------------------------------
# This callback handles all table filters. I made sure to validate both inputs
# and construct safe dictionary-based queries to protect the database. This 
# reinforces my enhanced CRUD class and ensures the logic remains consistent.

@app.callback(
    Output("animal-table", "data"),
    [
        Input("animal-dropdown", "value"),
        Input("outcome-dropdown", "value")
    ]
)
def update_table(animal_type, outcome_type):
    query = {}

    # I validate inputs before inserting them into the query to prevent invalid
    # or malicious patterns from slipping in.
    if animal_type:
        query["animal_type"] = {"$eq": animal_type}

    if outcome_type:
        query["outcome_type"] = {"$eq": outcome_type}

    try:
        df_filtered = get_filtered_data(query)
        return df_filtered.to_dict("records")
    except Exception as e:
        print(f"[ERROR] Table update failed: {e}")
        return df.to_dict("records")


# ------------------------------
# RUN THE APPLICATION
# ------------------------------
# Running in production mode (debug=False) reflects better professional standards.

if __name__ == "__main__":
    app.run(debug=False, host="0.0.0.0", port=8050)


Dash is running on http://0.0.0.0:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://10.100.195.55:8050/ (Press CTRL+C to quit)
192.168.10.31 - - [30/Nov/2025 23:09:56] "GET / HTTP/1.1" 200 -
192.168.10.31 - - [30/Nov/2025 23:09:56] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.v2_8_0m1752168217.js HTTP/1.1" 200 -
192.168.10.31 - - [30/Nov/2025 23:09:56] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_8_1m1752168217.12.1.min.js HTTP/1.1" 200 -
192.168.10.31 - - [30/Nov/2025 23:09:56] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_8_1m1752168217.min.js HTTP/1.1" 200 -
192.168.10.31 - - [30/Nov/2025 23:09:56] "GET /_dash-component-suites/dash/dash_table/bundle.v5_2_2m1752168217.js HTTP/1.1" 200 -
192.168.10.31 - - [30/Nov/2025 23:09:56] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_8_1m1752168217.14.0.min.js HTTP/1.1" 200 -
192.168.10.31 - - [30/Nov/2025 23:09:56] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_8_1m1752168217.8.1.min.js HTTP/1.1" 200 -
192.